## TransCoderで翻訳候補を生成するまでの一連の実装

### 環境の整備

#### ライブラリのインストール

In [ ]:
!pip install six
!pip install sacrebleu=="1.2.11"
!pip install submitit
!pip install clang
!pip install javalang

In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/tus_master_thesis

/content/drive/MyDrive/tus_master_thesis


In [3]:
#!git clone https://github.com/NVIDIA/apex.git
#!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
sys.path.append("/content/drive/MyDrive/tus_master_thesis/apex")

In [4]:
# !git clone https://github.com/glample/fastBPE.git
%cd fastBPE/
!python setup.py install
%cd ../

/content/drive/MyDrive/tus_master_thesis/fastBPE
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/install

In [5]:
# !git clone https://github.com/naru-99/lib763.git
%cd lib763
!bash install.sh
%cd ../

/content/drive/MyDrive/tus_master_thesis/lib763
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa

In [6]:
%cd /content/drive/MyDrive/tus_master_thesis/langMorphDA/experiment/transcoder
!git clone https://github.com/facebookresearch/TransCoder.git
!cp ./colab/UseTransCoder.py ./TransCoder/
!cp ./colab/extract_fgold_func.py ./TransCoder/

/content/drive/MyDrive/tus_master_thesis/langMorphDA/experiment/transcoder/TransCoder


#### ここで、一度再起動する
自作ライブラリが読み込めません;;

In [ ]:
%cd /content/drive/MyDrive/tus_master_thesis/langMorphDA/experiment/transcoder/TransCoder

### facebookresearch/TransCoderレポジトリのエラーに対処する

In [ ]:
!python patch_repo_error.py

In [ ]:
!python extract_fgold_func.py

In [ ]:
#!wget https://dl.fbaipublicfiles.com/transcoder/model_1.pth
#!wget https://dl.fbaipublicfiles.com/transcoder/model_2.pth

### 乱数のシード値を固定する

In [ ]:
import random
import numpy as np
import torch

def torch_fix_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True


torch_fix_seed(763)

In [9]:
!python UseTransCoder.py \
--src_lang python --tgt_lang java \
--model_path model_2.pth --beam_size 100 \
--save_dir save_dir --tgt_dir tgt_data/python_func

/content/drive/MyDrive/tus_master_thesis/langMorphDA/experiment/transcoder/TransCoder/UseTransCoder.py:171: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (os.path.isdir(params.save_dir), f"{params.save_dir} is not a directory")
/content/drive/MyDrive/tus_master_thesis/langMorphDA/experiment/transcoder/TransCoder/UseTransCoder.py:173: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (os.path.isdir(params.tgt_dir), f"{params.tgt_dir} is not a directory")
Loading codes from /content/drive/MyDrive/tus_master_thesis/langMorphDA/experiment/transcoder/TransCoder/data/BPE_with_comments_codes ...
Read 50000 codes from the codes file.


In [7]:
!python UseTransCoder.py \
--src_lang java --tgt_lang python \
--model_path model_1.pth --beam_size 100 \
--save_dir java2python --tgt_dir tgt_data/java_func

/content/drive/MyDrive/tus_master_thesis/langMorphDA/experiment/transcoder/TransCoder/UseTransCoder.py:171: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (os.path.isdir(params.save_dir), f"{params.save_dir} is not a directory")
/content/drive/MyDrive/tus_master_thesis/langMorphDA/experiment/transcoder/TransCoder/UseTransCoder.py:173: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (os.path.isdir(params.tgt_dir), f"{params.tgt_dir} is not a directory")
Loading codes from /content/drive/MyDrive/tus_master_thesis/langMorphDA/experiment/transcoder/TransCoder/data/BPE_with_comments_codes ...
Read 50000 codes from the codes file.
